In [89]:
import json
import os
from collections import defaultdict
import pandas as pd
import numpy as np

In [99]:
## USAGE EXTRACTION
read_folder = '../data/raw/chaos/'
write_folder = '../data/clean/'

In [129]:
usage_df = pd.DataFrame(columns=['pokemon', 'usage', 'month'])
for fp in sorted(os.listdir(read_folder)):
    try:
        with open(read_folder+fp, 'r') as f:
            d = json.load(f)

        month = fp[-7:-5] + '-' + fp[:4]
        to_save = defaultdict(list)

        for pokemon, d_vals in d['data'].items():

            to_save['pokemon'].append(pokemon)
            to_save['usage'].append(round(d_vals['usage'], 7))
            to_save['month'].append(month)

        usage_df = pd.concat([usage_df, pd.DataFrame(to_save)], sort=False, ignore_index=True)
    except:
        pass

In [123]:
usage_df.to_csv(write_folder+'usage.csv', header=True, index=False)